In [18]:
import pandas as pd
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import f1_score

Reading and Understanding data

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
pprint(vars(train))

{'_data': BlockManager
Items: Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')
Axis 1: RangeIndex(start=0, stop=7613, step=1)
IntBlock: slice(0, 8, 4), 2 x 7613, dtype: int64
ObjectBlock: slice(1, 4, 1), 3 x 7613, dtype: object,
 '_is_copy': None,
 '_item_cache': {}}


In [4]:
#train.head()
train["text"][1000]

'#OVOFest Drake straight body bagging Meek on that OVO stage. #ZIPHIMUP!'

In [5]:
sample.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


### Try k-nearest neighbour on the data

In [6]:
trainer,valid = train_test_split(train,
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [7]:
trainer.shape

(6090, 5)

In [8]:
tfv = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')

In [9]:
tfv.fit(list(trainer.text.values) + list(valid.text.values))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=3, ngram_range=(1, 3), norm='l2', preprocessor=None,
                smooth_idf=1, stop_words='english', strip_accents='unicode',
                sublinear_tf=1, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=1, vocabulary=None)

In [10]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [34]:
xtrain_tfv =  tfv.transform(trainer.text.values) 
xvalid_tfv = tfv.transform(valid.text.values)
pprint(vars(xtrain_tfv))

{'_shape': (6090, 9229),
 'data': array([0.33437892, 0.25069802, 0.06115396, ..., 0.31132424, 0.33215319,
       0.26934142]),
 'indices': array([8605, 8604, 7909, ..., 2824,  525,  524], dtype=int32),
 'indptr': array([    0,    15,    21, ..., 76789, 76810, 76825], dtype=int32),
 'maxprint': 50}


In [57]:
#clf = KNeighborsClassifier(n_neighbors=5)
#help(LogisticRegression)
clf = LogisticRegression(C=5, penalty='l1', solver='liblinear')
clf.fit(xtrain_tfv, trainer.target)
print("Number of features used:", np.sum(clf.coef_ != 0))

Number of features used: 1911


In [58]:
#predictions = clf.predict_proba(xvalid_tfv)
predictions =  clf.predict(xvalid_tfv)
print("Test set predictions:", predictions)

Test set predictions: [0 0 0 ... 1 1 1]


In [61]:
print("Training set accuracy: {:.2f}".format(clf.score(xtrain_tfv, trainer.target)))
print("Test set accuracy: {:.2f}".format(clf.score(xvalid_tfv, valid.target)))

Training set accuracy: 0.93
Test set accuracy: 0.78


In [59]:
train_predictions = clf.predict(xtrain_tfv)
print ("f1_score on training: %0.3f " % f1_score(trainer.target.values, train_predictions))

f1_score on training: 0.916 


In [60]:
#print ("logloss: %0.3f " % multiclass_logloss(valid.target.values, predictions))
print ("f1_score on test: %0.3f " % f1_score(valid.target.values, predictions))
#f1 score should be made as close to 1 as possible

f1_score on test: 0.727 


In [28]:
valid.target.values.shape[0]

1523

In [17]:
#help(LogisticRegression)